
*Задание 6

Используйте "Olivetti faces data-set from AT&T" из встроенных наборов данных библиотеки sklearn.
Обучить любую модель классификации на этом датасете до применения PCA (количество компонент подберите самостоятельно) и после него. Сравнить качество классификации по отложенной выборке.


In [148]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings('ignore')

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2
from sklearn.decomposition import PCA

In [22]:
def errors(Pred, Test):
    mse1 = mse(Pred, Test) 
    mae1 = mae(Pred, Test) 
    r2_1 = r2(Pred, Test)
    return {'mse': mse1, 'mae': mae1, 'r2': r2_1}


In [23]:
data = fetch_olivetti_faces(return_X_y = True)

In [62]:
X = data[0]
y = data[1]
X.shape

(400, 4096)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 91)

In [144]:
scaler = StandardScaler()

X_train_mms = scaler.fit_transform(X_train)

X_test_mms = scaler.transform(X_test)

scaler_rs = RobustScaler()

X_train_rs = scaler_rs.fit_transform(X_train)

X_test_rs = scaler_rs.transform(X_test)

In [134]:
svc = SVC(C = 100, gamma = 'auto')
svc.fit(X_train_mms, y_train)

SVC(C=100, gamma='auto')

In [149]:
y_pred = svc.predict(X_test_mms)
y_pred_tr = svc.predict(X_train_mms)
err_tst = accuracy_score(y_pred, y_test)

Применим PCA к нашим данным

In [116]:
#узнаем сколько главных компонент  с самой большой объяснённой дисперсией 
#нужно для сохранения 90, 95 и 99 процентов информации.
pca = PCA(random_state=100)
comp = []
pca.fit(X_train_mms)
for j in [0.9, 0.95, 0.99]:
    for i in range(len(pca.explained_variance_ratio_)):
        if pca.explained_variance_ratio_[:i].sum() > j:
            comp.append(i)
            break
print(comp)


[58, 101, 198]


In [150]:
pca_1 = PCA(n_components=comp[0], random_state=100)
X_train_mc_1 = pca_1.fit_transform(X_train_mms)
X_test_mc_1 = pca_1.transform(X_test_mms)

svc_1 = SVC(C= 100, gamma = 'auto')
svc_1.fit(X_train_mc_1, y_train)
y_pred_mc_1 = svc_1.predict(X_test_mc_1)
err_tst_mc_1 = accuracy_score(y_pred_mc_1, y_test)

In [151]:
pca_2 = PCA(n_components=comp[1], random_state=100)
X_train_mc_2 = pca_2.fit_transform(X_train_mms)
X_test_mc_2 = pca_2.transform(X_test_mms)

svc_2 = SVC(C = 100, gamma = 'auto')
svc_2.fit(X_train_mc_2, y_train)
y_pred_mc_2 = svc_2.predict(X_test_mc_2)
err_tst_mc_2 = accuracy_score(y_pred_mc_2, y_test)

In [152]:
pca_3 = PCA(n_components=comp[2], random_state=100)
X_train_mc_3 = pca_3.fit_transform(X_train_mms)
X_test_mc_3 = pca_3.transform(X_test_mms)

svc_3 = SVC(C = 100, gamma = 'auto')
svc_3.fit(X_train_mc_3, y_train)
y_pred_mc_3 = svc_3.predict(X_test_mc_3)
err_tst_mc_3 = accuracy_score(y_pred_mc_3, y_test)

Сравним величины ошибок

In [154]:
print('Тестовые данные')
print('Ошибка до PCA: {}' .format(err_tst))
print('Ошибка после PCA(90% сохраненной информации): {}' .format(err_tst_mc_1))
print('Ошибка после PCA(95% сохраненной информации): {}' .format(err_tst_mc_2))
print('Ошибка после PCA(95% сохраненной информации): {}' .format(err_tst_mc_3))

Тестовые данные
Ошибка до PCA: 0.925
Ошибка после PCA(90% сохраненной информации): 0.025
Ошибка после PCA(95% сохраненной информации): 0.025
Ошибка после PCA(95% сохраненной информации): 0.075


Видно, что для SVC сокращение признаков - критично. Попробуем KNN и оставим 99% информации

In [161]:
clf = KNeighborsClassifier(n_neighbors=6)

clf.fit(X_train_rs, y_train)

y_pred_knn = clf.predict(X_test_rs)
err_knn = accuracy_score(y_pred_knn, y_test)

In [162]:
pca_4 = PCA(n_components=comp[2], random_state=100)
X_train_rs_mc = pca_4.fit_transform(X_train_rs)
X_test_rs_mc = pca_4.transform(X_test_rs)

clf_1 = KNeighborsClassifier(n_neighbors=6)
clf_1.fit(X_train_rs_mc, y_train)
y_pred_mc_knn = clf_1.predict(X_test_rs_mc)
err_mc_knn = accuracy_score(y_pred_mc_knn, y_test)

In [164]:
pca_5 = PCA(n_components=comp[1], random_state=100)
X_train_rs_mc_1 = pca_5.fit_transform(X_train_rs)
X_test_rs_mc_1 = pca_5.transform(X_test_rs)

clf_2 = KNeighborsClassifier(n_neighbors=6)
clf_2.fit(X_train_rs_mc_1, y_train)
y_pred_mc_knn_1 = clf_2.predict(X_test_rs_mc_1)
err_mc_knn_1 = accuracy_score(y_pred_mc_knn_1, y_test)

In [165]:
print('Ошибка до PCA:{}'.format(err_knn) )
print('Ошибка после PCA(99% сохраненной информации): {}' .format(err_mc_knn))
print('Ошибка после PCA(95% сохраненной информации): {}' .format(err_mc_knn_1))

Ошибка до PCA:0.675
Ошибка после PCA(99% сохраненной информации): 0.6833333333333333
Ошибка после PCA(95% сохраненной информации): 0.6666666666666666


В случае сохранения 99% информации модель даже лучше, при 95% ошибка уже хуже